# Script pour manipuler un tirage et comparer des colonnes

In [2]:
# -*- coding: utf8 -*-
import pandas as pd
import pickle, glob
import itertools as it
import networkx as nx
import numpy as np

In [13]:
%matplotlib

Using matplotlib backend: MacOSX


In [3]:
phonologicalMap="-S"

In [57]:
def tiragesNom(nom):
    result=[tirage for tirage in listeTirages if "-"+nom in tirage]
    return result

def tiragesNomMorphomes(nom):
    result=[tirage for tirage in listeTiragesMorphomes if "-"+nom in tirage]
    return result
    

listeTirages=glob.glob("/Users/gilles/Box Sync/2015-Data/*-Tirage.pkl")
listeTirages200Mo=tiragesNom("200Mo")
listeTirages20=tiragesNom("20Mo")
listeTirages1=tiragesNom("1Mo")
listeTirages50k=tiragesNom("50Ko")
listeTirages100k=tiragesNom("100Ko")
listeTirages20000k=tiragesNom("20000Ko")
listeTiragesMorphomes=glob.glob("/Users/gilles/Box Sync/2015-Data/*-Tirage-*Morphomes.pkl")
listeTiragesMorphomes200Mo=tiragesNomMorphomes("200Mo")
listeTiragesMorphomes20=tiragesNomMorphomes("20Mo")
listeTiragesMorphomes1=tiragesNomMorphomes("1Mo")
listeTiragesMorphomes50k=tiragesNomMorphomes("50Ko")
listeTiragesMorphomes100k=tiragesNomMorphomes("100Ko")
listeTiragesMorphomes20000k=tiragesNomMorphomes("20000Ko")
listeTiragesMorphomes200Mo

['/Users/gilles/Box Sync/2015-Data/MGC-160628-0-200Mo-Tirage-Morphomes.pkl',
 '/Users/gilles/Box Sync/2015-Data/MGC-160628-0-200Mo-Tirage-S-170502-1511-Morphomes.pkl',
 '/Users/gilles/Box Sync/2015-Data/MGC-160716-0-200Mo-Tirage-Morphomes.pkl',
 '/Users/gilles/Box Sync/2015-Data/MGC-160716-1-200Mo-Tirage-Morphomes.pkl',
 '/Users/gilles/Box Sync/2015-Data/MGC-160716-2-200Mo-Tirage-Morphomes.pkl',
 '/Users/gilles/Box Sync/2015-Data/MGC-160717-0-200Mo-Tirage-Morphomes.pkl',
 '/Users/gilles/Box Sync/2015-Data/MGC-160718-0-200Mo-Tirage-Morphomes.pkl',
 '/Users/gilles/Box Sync/2015-Data/MGC-160719-0-200Mo-Tirage-Morphomes.pkl',
 '/Users/gilles/Box Sync/2015-Data/MGC-160720-0-200Mo-Tirage-Morphomes.pkl']

In [5]:
def lireLexique(nomLexique):
    try:
        with open(nomLexique, 'rb') as input:
            lexique=pickle.load(input)
    except:
        lexique=None
    return lexique

In [6]:
nomLexique=listeTirages1[0]
lexique=lireLexique(nomLexique)
lexique1=lexique[lexique["tir1"]>0]
taille=lexique[lexique["tir1"]>0]["tir1"].count()
paradigmes=pd.pivot_table(lexique[lexique["tir1"]>0], values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()


In [7]:
def compPaire(c1,c2):
    syncretisms=[]
    c1Val=paradigmes[c1].notnull()
    c2Val=paradigmes[c2].notnull()
    c1Sur=paradigmes[c1].str.contains(",")
    c2Sur=paradigmes[c2].str.contains(",")
    l1=len(paradigmes[paradigmes[c1].notnull()])
    l2=len(paradigmes[paradigmes[c2].notnull()])
    paire=paradigmes[c1Val & c2Val & (paradigmes[c1]!=paradigmes[c2])][[c1,c2]]
    lenDiff=len(paire[~paire[c1].str.contains(",") & ~paire[c2].str.contains(",")])
    if lenDiff>0:
        if debug:
            print u"%s ≠ %s"%(c1,c2)
            print "différence",lenDiff
            if lenDiff<12:
                print paire
    else:
        surAbondant=paire[paire[c1].str.contains(",") | paire[c2].str.contains(",")]
#            print "--------------------------------"
        if len(surAbondant)==0:
#                print u"%s = %s"%(c1,c2) 
            syncretisms.append(u"%s = %s"%(c1,c2))
        else:
            compatible=True
            for index,row in surAbondant.iterrows():
                if "," in row[c1]:
                    if "," in row[c2]:
                        if row[c1]!=row[c2]:
                            compatible=False
                    else:
                        if not row[c2] in row[c1].split(","):
                            compatible=False
                else:
                    if not row[c1] in row[c2].split(","):
                        compatible=False
            if compatible:
#                    print u"%s = %s"%(c1,c2)
                syncretisms.append(u"%s = %s"%(c1,c2))
            else:
                print u"%s ≠ %s"%(c1,c2)
                print surAbondant

In [8]:
def assignerTNS(row):
    if row[-2:] in [p+n for p in ["1","2","3"] for n in ["S","P"]]:
        return row[:2]
    else:
        return "NF"

def assignerPER(row):
    if row[-2:] in [p+n for n in ["S","P"] for p in ["1","2","3"] ]:
        return row[-2:]
    else:
        if row=="inf":
            return "1S"
        elif row=="pP":
            return "2S"
        elif row=="ppMS":
            return "3S"
        elif row=="ppMP":
            return "1P"
        elif row=="ppFS":
            return "2P"
        elif row=="ppFP":
            return "3P"

lexique["TNS"]=lexique["case"].apply(assignerTNS)
lexique["PER"]=lexique["case"].apply(assignerPER)

In [9]:
lexeme=u"être"
tableauTest=lexique[lexique["lexeme"]==lexeme].pivot_table(values='phono', index=['lexeme','TNS'], columns=["PER"], aggfunc=lambda x: ",".join(x))
tableauTest

PER            1P    1S     2P    2S    3P    3S
lexeme TNS                                      
être   NF    None   Etr   None   Etâ  None   Ete
       ai     fym    fy    fyt    fy   fyr    fy
       fi    s6rô  s6rE   s6re  s6ra  s6rô  s6ra
       ii    Etjô   EtE   Etje   EtE   EtE   EtE
       is   fysjô   fys  fysje   fys   fys    fy
       pI   swajô  None  swaje   swa  None  None
       pc   s6rjô  s6rE  s6rje  s6rE  s6rE  s6rE
       pi     sOm   sHi     Et     E    sô     E
       ps   swajô   swa  swaje   swa   swa   swa

In [10]:
ligTable=['pi', 'ii', 'fi','pc','ps','ai','is','pI','NF']
colTable=[p+n for n in ["S","P"] for p in ["1","2","3"] ]
colTuple=[("PER",col) for col in colTable]
ligTuple=[("TNS",lig) for lig in ligTable]

In [11]:
tableauTest=tableauTest.reindex(columns=colTable)

In [12]:
idxTableau = tableauTest.sortlevel(level='TNS').index
idxTableau=[idxTableau[i] for i in [7, 3, 2, 6, 1,4,5,0]]
tableauTest=tableauTest.reindex(idxTableau)
print tableauTest.to_latex().replace("None","")

\begin{tabular}{llllllll}
\toprule
     &    &    1S &    2S &    3S &     1P &     2P &    3P \\
lexeme & TNS &       &       &       &        &        &       \\
\midrule
être & pi &   sHi &     E &     E &    sOm &     Et &    sô \\
     & ii &   EtE &   EtE &   EtE &   Etjô &   Etje &   EtE \\
     & fi &  s6rE &  s6ra &  s6ra &   s6rô &   s6re &  s6rô \\
     & pc &  s6rE &  s6rE &  s6rE &  s6rjô &  s6rje &  s6rE \\
     & ai &    fy &    fy &    fy &    fym &    fyt &   fyr \\
     & is &   fys &   fys &    fy &  fysjô &  fysje &   fys \\
     & pI &   &   swa &   &  swajô &  swaje &   \\
     & NF &   Etr &   Etâ &   Ete &    &    &   \\
\bottomrule
\end{tabular}



# Nombre de formes différentes par taille d'échantillon

In [ ]:
unsPoints=[]
for tirage in listeTirages100k:
    lexique=lireLexique(tirage)
    tirs=lexique[lexique["tir1"]>0]["tir1"].sum()
    taille=lexique[lexique["tir1"]>0]["tir1"].count()
    unsPoints.append((tirs, taille)) 

In [ ]:
autresPoints=[(1000000,43563),
              (2000000,52654),
              (3000000,57648),
              (4000000,60647),
              (5000000,62926),
              (6000000,64507),
              (7000000,65694),
              (8000000,66597),
              (9000000,67395),
              (10000000,68049),
              (11000000,68590),
              (12000000,69007),
              (13000000,69382),
              (14000000,69690),
              (15000000,69959),
              (16000000,70178),
              (17000000,70379),
              (18000000,70547),
              (19000000,70704),
              (20000000,70859),  
              (200000000,74702),
              (400000000,76076),
              (600000000,77351),
              (800000000,78517),
              (1000000000,79565),
              (1200000000,80618),
              (1400000000,81578),
              (1600000000,82505),
             ]
points=autresPoints

In [ ]:
pdPoints=pd.DataFrame(points)
pdPoints.columns=["Sample Size", "Number of Forms"]
pdPoints.set_index("Sample Size",inplace=True)
xPoints=list(pdPoints.index)
pdPoints

In [ ]:
#pdPoints.plot(ylim=(0,300000))
pdPoints.plot(linewidth=5,marker="o",markersize=10,xlim=(0,2000e6))

# Nombre de lexèmes par nombre de formes par taille d'échantillon
- nom du fichier échantillon => tirage
- tirage de l'échantillon => lexique
- formes tirées de l'échantillon => lexique1
- cases tirées de l'échantillon => nbCases
- nombre de token dans le tirage =>nbTokens
- distribution du nombre de lexème par nombre de forme => distNbFormes
- liste des distributions => distributions


In [14]:
lexiquePrefix="MGC-160104"

with open("/Users/gilles/Box Sync/2015-Data/MGC-Vienna/"+lexiquePrefix+'-Verbes2-NbForms.pkl', 'rb') as input:
    dictNbFormes = pickle.load(input)
with open("/Users/gilles/Box Sync/2015-Data/MGC-Vienna/"+lexiquePrefix+'-Verbes2%s-Morphomes-NbForms.pkl'%phonologicalMap, 'rb') as input:
    dictNbFormesMorphomiques = pickle.load(input)

In [62]:
def nbMax(row):
    
    if brut:
        dictNb=dictNbFormes
    else:
        dictNb=dictNbFormesMorphomiques
        
    if dictNb[row["lexeme"]]==row["Number of Forms"]:
        return True
    else:
        return False

def nbFull(row):
    
    if brut:
        dictNb=dictNbFormes
        if dictNb[row["lexeme"]]==row["Number of Forms"] and row["Number of Forms"] in listeNbCasesMax:
            return True
        else:
            return False
    else:
        dictNb=dictNbFormesMorphomiques
        lexMax=dictNb[row["lexeme"]]
        if lexMax==31:
            if row["Number of Forms"]==min(listeNbCasesMax):
                return True
            elif row["Number of Forms"]>min(listeNbCasesMax):
                print row["lexeme"],"pb avec le nb de formes",row["Number of Forms"],"lexMax =",lexMax
                return False
            else:
                return False
        elif lexMax==32:
            if row["Number of Forms"]==max(listeNbCasesMax):
                return True
            elif row["Number of Forms"]>max(listeNbCasesMax):
                print row["lexeme"],"pb avec le nb de formes",row["Number of Forms"],"lexMax =",lexMax
                return False
            else:
                return False
        else:
            return False

In [64]:
distributions=[]
brut=False
for nTirage,tirage in enumerate(listeTiragesMorphomes20000k):
#    print tirage
    lexique=lireLexique(tirage)
    if isinstance(lexique, pd.DataFrame):
        lexique1=lexique[lexique["tir1"]>0]
        nbTokens=lexique1["tir1"].sum()
        nbCases=lexique1.groupby(by=["lexeme","case"],as_index=False).agg({"tir1":sum})
        listeMorphomes=lexique1.groupby(["case"],as_index=False)["lexeme"].count()["case"].tolist()
        nbMorphomes=len(listeMorphomes)
        listeMorphomesNoPP=[m for m in listeMorphomes if not "pp" in m]
        nbMorphomesOnePP=len(listeMorphomesNoPP)+1
        listeNbCasesMax=[nbMorphomesOnePP,nbMorphomes]
        print listeNbCasesMax
        nbFormes=nbCases.groupby(by=["lexeme"])[["case"]].count()
#        nbFormes=lexique1.groupby(by=["lexeme"])[["phono"]].count()
        nbFormes.columns=["Number of Forms"]
        distNbFormes=nbFormes.groupby(by=["Number of Forms"])[["Number of Forms"]].count()
        distNbFormes.columns=["Number of lexemes"]
        nbFormes.reset_index(inplace=True)
        lexemesMax=nbFormes[nbFormes.apply(nbMax,axis=1)]
        lexemesFull=nbFormes[nbFormes.apply(nbFull,axis=1)]
        distributions.append((nbTokens, distNbFormes,lexemesMax,lexemesFull))
        print tirage
        print "{:,}".format(nbTokens)
        print len(lexemesFull)
        print lexemesFull
        print distNbFormes[30:]
#        print lexemesMax
        print
        print
        if nTirage%1==0 and True:
            distNbFormes.plot(kind="bar",figsize=(20,10),ylim=(0,1000))

[30, 30]
/Users/gilles/Box Sync/2015-Data/MGC-170430-00-20000Ko-Tirage-S-170503-1907-Morphomes.pkl
1,000,000
0
Empty DataFrame
Columns: [lexeme, Number of Forms]
Index: []
Empty DataFrame
Columns: [Number of lexemes]
Index: []


[30, 31]
/Users/gilles/Box Sync/2015-Data/MGC-170430-01-20000Ko-Tirage-S-170503-1908-Morphomes.pkl
2,000,000
1
     lexeme  Number of Forms
2076  faire               31
Empty DataFrame
Columns: [Number of lexemes]
Index: []


[31, 32]
/Users/gilles/Box Sync/2015-Data/MGC-170430-02-20000Ko-Tirage-S-170503-1909-Morphomes.pkl
3,000,000
1
     lexeme  Number of Forms
2134  faire               32
                 Number of lexemes
Number of Forms                   
32                               1


[32, 33]
/Users/gilles/Box Sync/2015-Data/MGC-170430-03-20000Ko-Tirage-S-170503-1910-Morphomes.pkl
4,000,000
1
     lexeme  Number of Forms
2165  faire               33
                 Number of lexemes
Number of Forms                   
31                            

In [48]:
#nbCases[nbCases["lexeme"]=="faire"]
#lexique[lexique["lexeme"]=="faire"]
listeMorphomes=lexique1.groupby(["case"],as_index=False)["lexeme"].count()["case"].tolist()
nbMorphomes=len(listeMorphomes)
listeMorphomesNoPP=[m for m in listeMorphomes if not "pp" in m]
nbMorphomesOnePP=len(listeMorphomesNoPP)+1

In [ ]:
    
nbFormes.reset_index(inplace=True)
nbFormes[nbFormes.apply(nbMax,axis=1)]

# Calculs sur les tirages de 200Mo cumulés

In [ ]:
listeLexiques=listeTirages200
cumulLexique=lireLexique(listeLexiques[0])
cumulLexique["tir1"]=0
for nTirage,tirage in enumerate(listeLexiques):
#    print nTirage,
    lexique=lireLexique(tirage)
    cumulLexique["tir1"]=cumulLexique["tir1"]+lexique["tir1"]
    lexique1=cumulLexique[cumulLexique["tir1"]>0]
    nbTokens=lexique1["tir1"].sum()
    nbTypes=lexique1["tir1"].count()
    print "(%d,%d),"%(nbTokens,nbTypes)
    nbCases=lexique1.groupby(by=["lexeme","case"],as_index=False).agg({"tir1":sum})
    nbFormes=nbCases.groupby(by=["lexeme"])[["case"]].count()
    nbFormes.columns=["Number of Forms"]
    distNbFormes=nbFormes.groupby(by=["Number of Forms"])[["Number of Forms"]].count()
    distNbFormes.columns=["Number of lexemes"]
    distributions.append(distNbFormes)
    if nTirage%1==5:
        distNbFormes.plot(kind="bar",figsize=(20,10),ylim=(0,1000))    

In [ ]:
for n in [int(1e6*k) for k in range(1,1600)]:
    if not n in pdPoints.index:
        pdPoints.loc[n]=np.NaN
pdPoints.sort_index(inplace=True)
pdPoints.interpolate(method="slinear", order=4, inplace=True)
derivativeNb=pdPoints.diff().reset_index()
derivativeNb.set_index("Sample Size",inplace=True)

In [ ]:
funcDerivativeNb=derivativeNb[derivativeNb["Sample Size"].isin(xPoints)]
funcDerivativeNb.set_index("Sample Size",inplace=True)

In [ ]:
funcDerivativeNb.ix["150000000":].plot()

In [ ]:
funcDerivativeNb

In [ ]:
for tirage in listeTirages:
    lexique=lireLexique(tirage)
    if isinstance(lexique, pd.DataFrame):
        lexique1=lexique[lexique["tir1"]>0]
        tokens=lexique[lexique["tir1"]>0]["tir1"].sum()
        print tokens,tirage
        print


In [ ]:
lexique1.groupby(by=["lexeme"]).count()